## 필요한 라이브러리 Import 해주기

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import seaborn as sns 
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 

In [3]:
!nvidia-smi

Wed Apr 19 11:48:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.54       Driver Version: 510.54       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
|  0%   29C    P8    19W / 250W |   4461MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
|  0%   

## 필요한 데이터 로드해주기

In [2]:
train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/breastcancer_train_data.csv')
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/breastcancer_test_data.csv')

In [3]:
train_data.head(5)

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,50,2,1,1,0,1,3,1,1,40,1,1,13,1,50,0
1,59,2,1,0,0,0,1,3,1,12,1,1,11,2,99,0
2,44,2,1,0,0,0,0,2,1,11,1,1,8,2,77,0
3,59,2,0,1,1,2,0,2,1,25,1,0,32,5,107,0
4,69,2,1,0,0,0,3,1,1,18,1,1,4,1,48,0


In [4]:
test_data.head(5)

,Age,Race,Marital Status,T Stage,N Stage,6th Stage,differentiate,Grade,A Stage,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Survival Months,Status
0,68,2,1,1,1,2,3,1,1,42,1,1,25,5,48,0
1,62,2,1,2,0,2,0,2,1,70,1,0,3,3,6,1
2,57,2,1,1,1,2,0,2,1,22,1,1,16,5,57,0
3,41,2,1,1,0,1,1,3,1,28,1,1,8,1,103,0
4,65,2,0,1,0,1,0,2,1,46,1,1,3,1,84,0


In [5]:
train_stage_features=['A Stage','differentiate','Race','T Stage ','Grade','Estrogen Status','Progesterone Status','Marital Status','N Stage',
                      '6th Stage','Regional Node Examined','Tumor Size','Reginol Node Positive','Age','Survival Months']

In [6]:
test_stage_features=['A Stage','differentiate','Race','T Stage ','Grade','Estrogen Status','Progesterone Status','Marital Status','N Stage']

In [7]:
# train_X, new_test_data_X는 독립변수, train_y, new_test_data_y는 종속변수로 구성하기

train_X=train_data.loc[:,train_stage_features]
train_y=train_data['Status']
new_test_data_X=test_data.loc[:,test_stage_features]
new_test_data_y=test_data['Status']

In [8]:
# y label의 분포 확인하기

train_y.value_counts()

0    2726
1     493
Name: Status, dtype: int64

In [9]:
# Class weight을 해주기 위해 비율 구해주기 

ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

# Teacher Model

## 베이지안 최적화해주기 - LightgbmClassifier

**1. 5-fold 교차검증 이용해서 Train data로 Validation set을 Auc가 최고였을 때의 하이퍼파라미터 구하기**

**2. Learning rate 0.01~0.1, max_depth 3~9, n_estimators 100~1000이었을 때에서 가장 최적의 하이퍼파라미터 구하기**

**3. 부분 X로 앞에서 구한 train data의 Logit을 예측하는 분류모델 구하기**

In [11]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-14 00:25:29,728] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_135042/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-14 00:25:32,709] Trial 0 finished with value: 0.8536732821361168 and parameters: {'learning_rate': 0.01504880995917805, 'max_depth': 7, 'n_estimators': 293}. Best is trial 0 with value: 0.8536732821361168.
/tmp/ipykernel_135042/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-14 00:25:34,760] Trial 1 finished w

Best hyperparameters: {'learning_rate': 0.014830843941786215, 'max_depth': 3, 'n_estimators': 366}
Best AUC: 0.8710


In [10]:
# 앞에서 구한 최적의 하이퍼파라미터로 튜닝하기
lgb=lgb.LGBMClassifier(learning_rate=  0.014830843941786215,max_depth= 3,n_estimators=366,scale_pos_weight=ratio,random_state=42)

In [11]:
# 모델 학습하기
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.014830843941786215, max_depth=3,
               n_estimators=366, random_state=42,
               scale_pos_weight=5.529411764705882)

In [12]:
# train data의 독립변수들로 클래스의 예측확률값 구하기

train_prob=pd.DataFrame()
train_prob['prob']=lgb.predict_proba(train_X)[:,1]

In [13]:
# 오즈(odds) 구하기

train_prob['1_prob']=1-train_prob['prob']
train_prob['odds']=train_prob['prob']/train_prob['1_prob']

In [14]:
# 앞에서 구한 오즈로 로짓을 구하기

train_prob['loogits']=np.log(train_prob['odds'])
train_X_new=train_X[test_stage_features]

# Student Model

## 베이지안 최적화해주기 - Lightgbmregressor

**1. 5-fold 교차검증 이용해서 Train data로 Validation set을 RMSE가 최저였을 때의 하이퍼파라미터 구하기**

**2. Learning rate 0.01~0.1, max_depth 3~9, n_estimators 100~1000이었을 때에서 가장 최적의 하이퍼파라미터 구하기**

**3. 부분 X로 앞에서 구한 train data의 Logit을 예측하는 회귀모델 구하기**

In [17]:
import optuna
from lightgbm import LGBMRegressor

# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X_new, train_prob['loogits'],
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=50)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)

[I 2023-04-14 00:27:55,016] A new study created in memory with name: no-name-de676905-2481-4ed8-8757-171d9e61e9bc
/tmp/ipykernel_135042/4075368247.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-14 00:27:58,181] Trial 0 finished with value: 2.261252336427982 and parameters: {'n_estimators': 302, 'max_depth': 9, 'learning_rate': 0.013004485040227743}. Best is trial 0 with value: 2.261252336427982.
/tmp/ipykernel_135042/4075368247.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-1

{'n_estimators': 353, 'max_depth': 3, 'learning_rate': 0.011622195867547687}


In [18]:
## 최적의 하이퍼파라미터로 튜닝하기

model = LGBMRegressor(n_estimators=353,max_depth=3,learning_rate= 0.011622195867547687)

In [19]:
## 독립변수를 부분 X로 놓고 종속변수는 로짓으로 두고 학습하기

model.fit(train_X_new,train_prob['loogits'])

LGBMRegressor(learning_rate=0.011622195867547687, max_depth=3, n_estimators=353)

In [20]:
# model로 new_test_data_X로 predict하기

test_data=pd.DataFrame()
test_data['loogits']=model.predict(new_test_data_X)

In [21]:
# loogits으로 test data의 예측확률 값을 구하기

test_data['new_prob']=1/(1+np.exp(-test_data['loogits']))

In [44]:
# 정확도, 정밀도, 재현율, 혼동행렬, F-1 Score, 

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [45]:
get_clf_prob(new_test_data_y,test_data['new_prob'])

임계값:  0.5
오차행렬
[[599  83]
 [ 80  43]]
정확도: 0.7975, 정밀도: 0.3413, 재현율: 0.3496, F1:0.6128, AUC: 0.6902
